In [ ]:
import json
import csv
import numpy as np
import pandas as pd
from urllib.parse import quote  
import scipy.stats as stats
import math
from difflib import SequenceMatcher
from nltk import everygrams

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

In [ ]:
data = pd.read_csv(f'PATH/final_version.csv.gz',compression='gzip',lineterminator='\n');
data.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,time_zone,location,default_profile,default_profile_image,geo_enabled,profile_image_url,profile_banner_url,profile_use_background_image,profile_background_image_url_https,profile_text_color,profile_image_url_https,profile_sidebar_border_color,profile_background_tile,profile_sidebar_fill_color,profile_background_image_url,profile_background_color,profile_link_color,utc_offset,is_translator,follow_request_sent,protected,verified,notifications,description,contributors_enabled,following,created_at,updated,tweets_created_at,diff_days,class
0,78740551,jylesza needham,greymouthjylesz,252,32,606,3,1,NaN,en,Auckland,NaN,NaN,NaN,1.0,http://pbs.twimg.com/profile_images/2277243297...,NaN,1.0,https://abs.twimg.com/images/themes/theme1/bg.png,333333,https://pbs.twimg.com/profile_images/227724329...,C0DEED,NaN,DDEEF6,http://abs.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,43200.0,NaN,NaN,NaN,NaN,NaN,my name is jylesza and i am from new zealand a...,NaN,NaN,Wed Sep 30 23:34:09 +0000 2009,2016-03-15 16:05:07,NaN,1.0,human
1,78468090,Adie Austria,adie_austria,2,15,701,0,0,NaN,en,Baghdad,NaN,1.0,NaN,NaN,http://a0.twimg.com/profile_images/2354675105/...,NaN,1.0,https://si0.twimg.com/images/themes/theme1/bg.png,333333,https://si0.twimg.com/profile_images/235467510...,C0DEED,NaN,DDEEF6,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,10800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wed Sep 30 00:54:11 +0000 2009,2013-06-12 18:38:35,Fri Apr 26 09:49:02 +0000 2013,1304.0,bot
2,381753637,Anafan2,Anafan2,23799,462,272,4116,10,http://t.co/YfWIseUpdQ,en,NaN,"here, there, everywhere",1.0,NaN,NaN,http://pbs.twimg.com/profile_images/5928642758...,https://pbs.twimg.com/profile_banners/38175363...,1.0,https://abs.twimg.com/images/themes/theme1/bg.png,333333,https://pbs.twimg.com/profile_images/592864275...,C0DEED,NaN,DDEEF6,http://abs.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,NaN,NaN,NaN,NaN,NaN,NaN,"Loca fan de la Tiuchis,OUAT fan.proud of being...",NaN,NaN,Wed Sep 28 21:46:53 +0000 2011,2016-03-15 15:54:20,NaN,1.0,human
3,381715808,"-,'new',- Auau",Winornera,54639,1307,319,6893,12,http://t.co/lZOISTowhH,en,Central Time (US & Canada),Texas,NaN,NaN,NaN,http://pbs.twimg.com/profile_images/5942071807...,https://pbs.twimg.com/profile_banners/38171580...,1.0,https://pbs.twimg.com/profile_background_image...,362720,https://pbs.twimg.com/profile_images/594207180...,000000,1.0,E5507E,http://pbs.twimg.com/profile_background_images...,131516,FA743E,-18000.0,NaN,NaN,NaN,NaN,NaN,Lesb|Artist|Writer|Short|♪My lingerie and cute...,NaN,NaN,Wed Sep 28 20:20:29 +0000 2011,2016-03-15 16:01:35,NaN,1.0,human
4,381602776,Alan Ralph,moodygills,3393,206,207,4510,1,NaN,en,Pacific Time (US & Canada),NaN,NaN,NaN,1.0,http://pbs.twimg.com/profile_images/5935875343...,https://pbs.twimg.com/profile_banners/38160277...,1.0,https://abs.twimg.com/images/themes/theme15/bg...,333333,https://pbs.twimg.com/profile_images/593587534...,FFFFFF,NaN,C0DFEC,http://abs.twimg.com/images/themes/theme15/bg.png,022330,0084B4,-25200.0,NaN,NaN,NaN,NaN,NaN,San leandro✈️ Instagram: benz_boy510,NaN,NaN,Wed Sep 28 16:24:08 +0000 2011,2016-03-15 16:08:01,NaN,1.0,human


In [ ]:
def likehood(s):
  Bi_list=["".join(item) for item in list(everygrams(s, 2, 2))]
  K=sum(list({i:Bi_list.count(i) for i in Bi_list}.values()))
  CBi= len(np.unique(Bi_list, return_counts=True)[1])
  return CBi/K if K>0 else 1
  
def entropy(string):
    prob = [ float(string.count(c)) / len(string) for c in dict.fromkeys(list(string)) ]
    entropy = - sum([ p * math.log(p) / math.log(2.0) for p in prob ])
    return entropy/len(string) if len(string)>0 else 1
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
data["name"].fillna("",inplace=True)
data["description"].fillna("",inplace=True)
data['name_length']=data['name'].map(lambda x: len(x))
data['screen_name_length']=data['screen_name'].map(lambda x: len(str(x)))
data['description_length']=data['description'].map(lambda x: len(x))

data['num_digits_in_name']=data['name'].map(lambda x: sum(c.isdigit() for c in x))
data['num_digits_in_screen_name']=data['screen_name'].map(lambda x: sum(c.isdigit() for c in str(x)))

data['tweet_freq']=data['statuses_count']/data['diff_days']
data['followers_growth_rate']=data['followers_count']/data['diff_days']
data['friends_growth_rate']=data['friends_count']/data['diff_days']
data['favourites_growth_rate']=data['favourites_count']/data['diff_days']
data['listed_growth_rate']=data['listed_count']/data['diff_days']
data['followers_friends_ratio']=data['followers_count']/data['diff_days']

data['Name_entropy']=data.apply(lambda row: entropy(str(row['name'])),axis=1)
data['Screen_name_entropy']=data.apply(lambda row: entropy(str(row['screen_name'])),axis=1)
data['Screen_name_freq']=data.apply(lambda row: likehood(str(row['screen_name'])),axis=1)
data['Name_similarity']=data.apply(lambda row: similar(str(row['screen_name']),str(row['name'])),axis=1)
data['Name_freq']=data.apply(lambda row: likehood(str(row['name'])) if len(row['name'])>0 else 1,axis=1)

In [ ]:
columns=[
    'id', 'name', 'screen_name', 'statuses_count', 'followers_count', 'friends_count', 'favourites_count', 
    'listed_count', 'url', 'lang', 'time_zone', 'location', 'default_profile', 'default_profile_image', 
    'geo_enabled', 'profile_image_url', 'profile_banner_url', 'profile_use_background_image', 
    'profile_background_image_url_https', 'profile_text_color', 'profile_image_url_https', 'profile_sidebar_border_color', 
    'profile_background_tile', 'profile_sidebar_fill_color', 'profile_background_image_url', 'profile_background_color', 
    'profile_link_color', 'utc_offset', 'is_translator', 'follow_request_sent', 'protected', 'verified', 'notifications', 
    'description', 'contributors_enabled', 'following', 'created_at', 'updated'
]

In [ ]:
data.drop(columns=[
        "lang","time_zone","follow_request_sent","utc_offset",
        "notifications","following","protected","contributors_enabled",
        "is_translator","profile_sidebar_fill_color","profile_link_color","profile_sidebar_border_color","profile_background_color","profile_text_color",
         "id","tweets_created_at","created_at","profile_image_url_https","profile_image_url",
         "profile_background_image_url","updated"
], axis=1, inplace=True);

In [ ]:
data["has_extended_profile"]=False
changeColumns=["profile_background_tile","default_profile","profile_use_background_image","default_profile_image","geo_enabled","verified","has_extended_profile"]
data[changeColumns]= data[changeColumns].applymap({True: 1, False: 0}.get)

# The same solution but it work for one columns
data['profile_background_tile'] = data['profile_background_tile'].map({True: 1, False: 0});

data["profile_background_image_url_https"]=np.where(data["profile_background_image_url_https"].isnull(),0,1)
data["_url"]=np.where(data["url"].isnull(),0,1)
data["_location"]=np.where(data["location"].isnull(),0,1)
data["profile_banner_url"]=np.where(data["profile_banner_url"].isnull(),0,1)
data["entities"]=0
data.drop(columns=["url","location","name","description","screen_name"], axis=1, inplace=True);

In [ ]:
data.nunique(axis=0,dropna=False)

statuses_count                        3946
followers_count                       2060
friends_count                         2093
favourites_count                      2454
listed_count                           190
default_profile                          2
default_profile_image                    2
geo_enabled                              2
profile_banner_url                       2
profile_use_background_image             2
profile_background_image_url_https       2
profile_background_tile                  1
verified                                 2
diff_days                             1338
class                                    2
name_length                             28
screen_name_length                      13
description_length                     167
num_digits_in_name                       9
num_digits_in_screen_name               12
tweet_freq                            7643
followers_growth_rate                 5691
friends_growth_rate                   7268
favourites_

In [ ]:
removeNaN=["profile_background_tile","default_profile","profile_use_background_image","default_profile_image","geo_enabled","verified"]
data[removeNaN]= data[removeNaN].applymap(lambda x: 0 if pd.isnull(x) else x)

In [ ]:
data.isnull().any()

statuses_count                        False
followers_count                       False
friends_count                         False
favourites_count                      False
listed_count                          False
default_profile                       False
default_profile_image                 False
geo_enabled                           False
profile_banner_url                    False
profile_use_background_image          False
profile_background_image_url_https    False
profile_background_tile               False
verified                              False
diff_days                             False
class                                 False
name_length                           False
screen_name_length                    False
description_length                    False
num_digits_in_name                    False
num_digits_in_screen_name             False
tweet_freq                            False
followers_growth_rate                 False
friends_growth_rate             

In [ ]:
columnList=[
    'listed_count','statuses_count','favourites_count','profile_background_tile','profile_background_image_url_https','friends_count',
    'has_extended_profile','followers_count','default_profile','profile_use_background_image','default_profile_image','profile_banner_url',
    'geo_enabled','verified','entities','class','diff_days','name_length','screen_name_length','description_length','num_digits_in_name',
    'num_digits_in_screen_name','tweet_freq','followers_growth_rate','friends_growth_rate','favourites_growth_rate','listed_growth_rate',
    'followers_friends_ratio','Name_entropy','Screen_name_entropy','Name_freq','Screen_name_freq','Name_similarity','_url','_location'
]
data=data[columnList]

In [ ]:
data.head()

,listed_count,statuses_count,favourites_count,profile_background_tile,profile_background_image_url_https,friends_count,has_extended_profile,followers_count,default_profile,profile_use_background_image,default_profile_image,profile_banner_url,geo_enabled,verified,entities,class,diff_days,name_length,screen_name_length,description_length,num_digits_in_name,num_digits_in_screen_name,tweet_freq,followers_growth_rate,friends_growth_rate,favourites_growth_rate,listed_growth_rate,followers_friends_ratio,Name_entropy,Screen_name_entropy,Name_freq,Screen_name_freq,Name_similarity,_url,_location
0,1,252,3,0,1,606,0,32,0.0,1.0,0.0,0,1.0,0.0,0,human,1.0,15,15,60,0,0,252.000000,32.000000,606.000000,3.0,1.0,32.000000,0.230438,0.242682,1.0,1.0,0.400000,0,0
1,0,2,0,0,1,701,0,15,1.0,1.0,0.0,0,0.0,0.0,0,bot,1304.0,12,12,0,0,0,0.001534,0.011503,0.537577,0.0,0.0,0.011503,0.270969,0.251838,1.0,1.0,0.750000,0,0
2,10,23799,4116,0,1,272,0,462,1.0,1.0,0.0,1,0.0,0.0,0,human,1.0,8,7,143,1,1,23799.000000,462.000000,272.000000,4116.0,10.0,462.000000,0.312500,0.319418,1.0,1.0,0.933333,1,1
3,12,54639,6893,0,1,319,0,1307,0.0,1.0,0.0,1,0.0,0.0,0,human,1.0,14,9,106,0,0,54639.000000,1307.000000,319.000000,6893.0,12.0,1307.000000,0.231138,0.302831,1.0,1.0,0.260870,1,1
4,1,3393,4510,0,1,207,0,206,0.0,1.0,0.0,1,1.0,0.0,0,human,1.0,10,10,36,0,0,3393.000000,206.000000,207.000000,4510.0,1.0,206.000000,0.292193,0.292193,1.0,1.0,0.200000,0,0


In [ ]:
data.to_csv(f'PATH/DS16.csv.gz',compression='gzip',index=False);